In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
import time
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from swipes.isosurface import isosurface
import matplotlib.tri as mtri
from skimage import measure

import matplotlib
%matplotlib notebook

# Circle calculation

In [ ]:
# Generate a level set about zero of two identical ellipsoids in 3D
ellip_base = ellipsoid(5,5,5, levelset=True)


# Use marching cubes to obtain the surface mesh of these ellipsoids
verts, faces, normals, values = measure.marching_cubes_lewiner(ellip_base, 0,use_classic=True)


# Display resulting triangular mesh using Matplotlib. This can also be done
# with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')


# Fancy indexing: `verts[faces]` to generate a collection of triangles
mesh = Poly3DCollection(verts[faces])
mesh.set_edgecolor('k')
ax.add_collection3d(mesh)


ax.set_xlabel("x-axis: a = 6 per ellipsoid")
ax.set_ylabel("y-axis: b = 10")
ax.set_zlabel("z-axis: c = 16")


ax.set_xlim(0, 12)  # a = 6 (times two for 2nd ellipsoid)
ax.set_ylim(0, 12)  # b = 10
ax.set_zlim(0, 12)  # c = 16

plt.tight_layout()
plt.show()

In [ ]:
# calculate triangle areas
points = verts[faces]
side1 = points[:,:,1] - points[:,:,0]
side2 = points[:,:,0] - points[:,:,2]
area = np.sqrt((np.cross(side1,side2)**2).sum(axis=1))/2


vertex_neighbor_area = np.zeros((verts.shape[0],))
vertex_matrix = np.zeros((verts.shape[0],3,3))

for (i,val) in enumerate(points):
    # index of the triangles in the vertex data
    t = faces[i]
    A = area[i]
    print(i)
    if A!= 0:
        for j in range(3):
            norm = normals[t[j]]
            vert = verts[t[j]]
            dyad = np.outer(norm, norm)
            tangential = np.eye(3) - dyad
            for k in range(3):
                if (j != k):
                    diff = vert - verts[t[k]]
                    diffsq = (diff**2).sum()

                    k = 2* np.dot(diff, norm)/diffsq
                    T = np.dot(tangential, diff)
                    T = T/np.sqrt((T**2).sum())

                    matrix = A*k*np.outer(T,T)
                    vertex_matrix[t[j]] += matrix         
                    vertex_neighbor_area[t[j]] += A

In [ ]:
ix = 156

q = vertex_matrix[ix]/vertex_neighbor_area[ix]
print(q)
E = np.array([1,0,0])

norm1 = ((E - normals[ix])**2).sum()
norm2 = ((E + normals[ix])**2).sum()

if (norm2 > norm1):
    sign = 1
else:
    sign = -1
    
W = E + sign*normals[ix]
W = W / np.sqrt((W**2).sum())
Q = np.eye(3) - 2*np.outer(W,W)
l = Q.T.dot(q).dot(Q)
sub = l[1:,1:]
eig,eigv = np.linalg.eig(sub)
k1 = 3*eig[1] - eig[0]
k2 = 3*eig[0] - eig[1]
print("k1 = ", k1)
print("k2 = ", k2)
print("mean curvature = ", 1/2*(k1+k2))

# AFP calculations

In [ ]:
f = open("test.out","r")
lines = f.readlines()
data = lines[0].split()
data = [float(a) for a in data]
f.close()
fielddata = data

f = open("vertex.out","r")
lines = f.readlines()
lines = [l.lstrip().rstrip("\n").split() for l in lines]
lines = np.array([[float(k) for k in l] for l in lines])
f.close()
vertices = lines

f = open("triangle.out","r")
lines = f.readlines()
lines = [l.lstrip().rstrip("\n").split() for l in lines]
lines = np.array([[int(k) for k in l] for l in lines])
f.close()
triangle_indices = lines

f = open("normals.out","r")
lines = f.readlines()
lines = [[float(f) for f in l.lstrip("\n").split()] for l in lines]
f.close()
normals = np.array(lines)

f = open("curve.out","r")
lines = f.readlines()
lines = [float(l.lstrip("\n")) for l in lines]
f.close()
curvedata = np.array(lines)

In [ ]:
Nx, Ny, Nz = 50, 50, 50
field = np.zeros((Nx, Ny, Nz))
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            field[i,j,k] = fielddata[k*Nx*Ny + j*Nx + i]

In [ ]:
points = vertices[triangle_indices]
# Display resulting triangular mesh using Matplotlib. This can also be done
# with mayavi (see skimage.measure.marching_cubes_lewiner docstring).
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Fancy indexing: `verts[faces]` to generate a collection of triangles
mesh = Poly3DCollection(points)
mesh.set_edgecolor('k')
ax.add_collection3d(mesh)

ax.set_xlabel("x-axis: a = 6 per ellipsoid")
ax.set_ylabel("y-axis: b = 10")
ax.set_zlabel("z-axis: c = 16")

ax.set_xlim(0, 10)  # a = 6 (times two for 2nd ellipsoid)
ax.set_ylim(0, 10)  # b = 10
ax.set_zlim(0, 10)  # c = 16

plt.tight_layout()
plt.show()

In [ ]:
idx = np.argwhere(vertices[:,0] >3.2)
idx = idx.flatten()
curve_select = curvedata[idx]
vert_select = vertices[idx]

points = vertices[triangle_indices]
p = points.reshape(-1,3)

fig = plt.figure()
ax = plt.axes(projection ='3d')  

m = ax.scatter(vert_select[:,0],vert_select[:,1],vert_select[:,2],vmin=0, vmax=5, c=np.abs(curve_select),cmap='RdBu')
ax.set_xlim(0,10)
cbar = fig.colorbar(m)

In [ ]:
neighbors = [[] for _ in range(vertices.shape[0])]
curvatureVal = [[] for _ in range(vertices.shape[0])]
curvaturevalue = np.ones((vertices.shape[0],))
neighbornums = np.zeros((vertices.shape[0],))

for i in range(triangle_indices.shape[0]):
    t = triangle_indices[i]
    
    for j in range(3):
        i1 = t[j]
        for k in range(j+1,3):
            i2 = t[k]
            if i2 not in neighbors[i1]:
                diff = vertices[i1] - vertices[i2]
                diffsq = (diff**2).sum()

                n1 = normals[i1]
                n2 = normals[i2]

                diffn = n1 - n2

                dotproduct = np.dot(diffn, diff)
                val = dotproduct/diffsq

                curvatureVal[i1].append(val)
                curvatureVal[i2].append(val)
                neighbors[i1].append(i2)
                neighbors[i2].append(i1)
                neighbornums[i1] += 1
                neighbornums[i2] += 1
                curvaturevalue[i1] *= abs(val)
                curvaturevalue[i2] *= abs(val)
curvaturevalue = curvaturevalue**(1/neighbornums)

In [ ]:
ix = np.argwhere(vertices[:,0] > 3.2)
c = curvaturevalue[ix]

In [ ]:
curvatureVal

In [ ]:
triangle_indices[12270]

In [ ]:
np.argwhere(triangle_indices == 6236)

In [ ]:
vertices[6236]

In [ ]:
normals[6236]

In [ ]:
# calculate triangle areas
points = vertices[triangle_indices]
side1 = points[:,:,1] - points[:,:,0]
side2 = points[:,:,0] - points[:,:,2]
area = np.sqrt((np.cross(side1,side2)**2).sum(axis=1))/2


vertex_neighbor_area = np.zeros((vertices.shape[0],))
vertex_matrix = np.zeros((vertices.shape[0],3,3))

for (i,val) in enumerate(points):
    # index of the triangles in the vertex data
    t = triangle_indices[i]
    A = area[i]
    print(i)
    for j in range(3):
        norm = normals[t[j]]
        vert = vertices[t[j]]
        dyad = np.outer(norm, norm)
        tangential = np.eye(3) - dyad
        for k in range(3):
            if (j != k):
                diff = vert - vertices[t[k]]
                diffsq = (diff**2).sum()
                
                k = 2* np.dot(diff, norm)/diffsq
                T = np.dot(tangential, diff)
                T = T/np.sqrt((T**2).sum())
                
                matrix = A*k*np.outer(T,T)
                vertex_matrix[t[j]] += matrix         
                vertex_neighbor_area[t[j]] += A

In [ ]:
idx[50]

In [ ]:
vertices[50]

In [ ]:
idx = np.argwhere(vertices[:,0] > 7)

In [ ]:
ix = 50

l = vertex_matrix[ix]/vertex_neighbor_area[ix]
x = np.array([1,0,0])
E1 = (x - normals[ix])
E1 = E1/np.sqrt((E1**2).sum())
Q = np.eye(3) - 2*np.outer(E1,E1)
res = (Q.T).dot(l).dot(Q)
eigv, eigvec = np.linalg.eig(res[1:,1:])
k1 = 3*eigv[0] - eigv[1]
k2 = 3*eigv[1] - eigv[0]
print("k1 = ", k1)
print("k2 = ", k2)

In [ ]:
np.linalg.eig(l)